# Scale generator for SunVox

This is a series of note translators for SunVox that lock your notes
into a specific scale.

It requires SunVox 1.9.2 or higher to use effectively.

## How to use them

In your existing project, use the "Merge project" command from the
SunVox main menu, then select the scale you want to use.

A chain of new modules will appear. You may wish to rearrange them
to suit your needs.

Connect the `O ...` module to the synth you want to send notes to.
Then, send notes to the `I ...` module at the other side of the chain.

## How it works

[*musthe*](https://github.com/gciruelos/musthe) provides a basic music theory engine. We maintain fork in the same directory as this notebook.

[*Radiant Voices*](https://radiant-voices.readthedocs.io/) lets us construct and write SunVox compatible project files.

The code below goes through all notes and scales available in *musthe*, and creates the structures needed to lock notes into those scales.

In [1]:
import musthe as m
import rv.api as rv

In [2]:
def scale_project(root_note, scale_name):
    name = '{} {} scale'.format(root_note, scale_name).replace('_', ' ')
    filename = '{}.sunvox'.format(name)
    
    scale = m.scale(m.Note(root_note), scale_name=scale_name)
    scale_ids = sorted({n.note_id for n in scale})
    
    proj = rv.Project()
    proj.name = name
    in_mod = proj.new_module(rv.m.MultiSynth, name='I {}'.format(name))
    in_mod.color = (0, 255, 0)
    out_mod = proj.new_module(rv.m.MultiSynth, name='O {}'.format(name))
    out_mod.color = (0, 0, 255)
    gate_mods = {}

    def gate_mod(offset):
        if offset not in gate_mods:
            gate = gate_mods[offset] = proj.new_module(
                rv.m.MultiSynth, 
                name='G{} {}'.format(offset, name),
            )
            gate.color = (64, 64, 64)
            if offset > 0:
                transpose = proj.new_module(
                    rv.m.MultiSynth,
                    name='T{} {}'.format(offset, name),
                    transpose=-offset,
                )
                transpose.color = (64, 64, 64)
                in_mod >> gate >> transpose >> out_mod
            else:
                in_mod >> gate >> out_mod
            for x in range(128):
                gate.nv_curve.values[x] = 0
        return gate_mods[offset]

    # Start one octave below to ensure offset is initialized
    # once we get to the note range we care about.
    offset = 0
    for note in range(-12, 128):
        if note % 12 in scale_ids:
            offset = 0
        else:
            offset += 1
        gate = gate_mod(offset)
        if note >= 0:
            gate.nv_curve.values[note] = 255

    proj.in_mod = in_mod
            
    proj.layout(prog='dot')            
    return proj, filename

In [3]:
for root_note in ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']:
    root_note_id = m.Note(root_note).note_id
    for scale_name in [
        'major',
        'natural_minor',
        'harmonic_minor',
        'melodic_minor',
        'dorian',
        'locrian',
        'lydian',
        'mixolydian',
        'phrygian',
        'major_pentatonic',
        'minor_pentatonic',
    ]:
        try:
            proj, filename = scale_project(root_note, scale_name)
            filename = filename.replace('#', 's')
        except:
            print('***', root_note, scale_name)
        else:
            with open(filename, 'wb') as f:
                proj.write_to(f)
            print(':-)', filename)
            if root_note_id:
                filename = 'Transposed to C - {}'.format(filename)
                proj.in_mod.transpose = -root_note_id
                with open(filename, 'wb') as f:
                    proj.write_to(f)

:-) C major scale.sunvox
:-) C natural minor scale.sunvox
:-) C harmonic minor scale.sunvox
:-) C melodic minor scale.sunvox
:-) C dorian scale.sunvox
:-) C locrian scale.sunvox
:-) C lydian scale.sunvox
:-) C mixolydian scale.sunvox
:-) C phrygian scale.sunvox
:-) C major pentatonic scale.sunvox
:-) C minor pentatonic scale.sunvox
:-) Cs major scale.sunvox
:-) Cs natural minor scale.sunvox
:-) Cs harmonic minor scale.sunvox
:-) Cs melodic minor scale.sunvox
:-) Cs dorian scale.sunvox
:-) Cs locrian scale.sunvox
:-) Cs lydian scale.sunvox
:-) Cs mixolydian scale.sunvox
:-) Cs phrygian scale.sunvox
:-) Cs major pentatonic scale.sunvox
:-) Cs minor pentatonic scale.sunvox
:-) Db major scale.sunvox
*** Db natural_minor
:-) Db harmonic minor scale.sunvox
:-) Db melodic minor scale.sunvox
*** Db dorian
*** Db locrian
:-) Db lydian scale.sunvox
*** Db mixolydian
*** Db phrygian
:-) Db major pentatonic scale.sunvox
*** Db minor_pentatonic
:-) D major scale.sunvox
:-) D natural minor scale.sun